 # GAD (Genetic Association Database)

 Author: Moshe Silverstein <br/>
 Date: 8-17 <br/>
 Data Source: https://geneticassociationdb.nih.gov/

 Reviewer: Charles Dai <br>
 Updated: 6-20

In [1]:
# appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [2]:
import sys
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import utility_functions as uf
import lookup

In [3]:
# from clustergrammer_widget import *
# net = Network(clustergrammer_widget)

In [4]:
%load_ext autoreload
%autoreload 2

 ### Python Version

In [5]:
sys.version

'3.8.0 (default, Oct 28 2019, 16:14:01) \n[GCC 8.3.0]'

 # Initialization

 ### Choose Attribute

In [14]:
%%appyter code_eval

{% set attribute = ChoiceField(
    name='attribute',
    label='Attribute',
    choices={
        'Disease': "'DISEASE'",
        'Disease Class': "'DIS_CLASS'"
    },
    default='Disease',
    section='data'
) %}

 ### Load Mapping Dictionaries

In [7]:
symbol_lookup, geneid_lookup = lookup.get_lookups()

Gathering sources: 100%|██████████| 3/3 [00:09<00:00,  3.25s/it]


 ### Output Path

In [15]:
%%appyter code_exec

output_name = 'gad-' + {{attribute}}.lower()

path = 'Output/GAD-' + {{attribute}}.replace('_', '-')
if not os.path.exists(path):
    os.makedirs(path)

```python
output_name = 'gad-' + 'DISEASE'.lower()
path = 'Output/GAD-' + 'DISEASE'.replace('_', '-')
if not os.path.exists(path):
    os.makedirs(path)
```

In [16]:
%%appyter hide_code
{% do SectionField(
    name='data',
    title='Load Data',
    subtitle='Upload Files from the Gene Assocation Database',
) %}

 # Load Data

In [20]:
%%appyter code_exec

df = pd.read_csv({{FileField(
    constraint='.*\.tsv$',
    name='gadcdc_data', 
    label='GAD CDC Data', 
    default='Input/GAD/GADCDC_data.tsv',
    section='data')
}}, sep='\t', usecols=['GENE', {{attribute}}], index_col='GENE')

```python

df = pd.read_csv('Input/GAD/GADCDC_data.tsv', sep='\t', usecols=['GENE', 'DISEASE'], index_col='GENE')
```

In [21]:
df.head()

,DISEASE
GENE,
HLA-A,leukemia
HLA-A,leukemia
HLA-A,alopecia areata
HLA-A,leukemia
HLA-A,melanoma


In [22]:
df.shape

(167132, 1)

 # Filter Data

 ## Map Gene Symbols to Up-to-date Approved Gene Symbols

In [23]:
df = uf.mapgenesymbols(df, symbol_lookup)
df.shape

100%|██████████| 167132/167132 [00:00<00:00, 717541.02it/s]


(163760, 1)

 # Analyze Data

 ## Create Binary Matrix

In [ ]:
binary_matrix = uf.createBinaryMatrix(df)
binary_matrix.head()

In [ ]:
binary_matrix.shape

In [ ]:
uf.saveData(binary_matrix, path, output_name + '_binary_matrix', 
            compression='npz', dtype=np.uint8)

 ## Create Gene List

In [ ]:
gene_list = uf.createGeneList(binary_matrix, geneid_lookup)
gene_list.head()

In [ ]:
gene_list.shape

In [ ]:
uf.saveData(gene_list, path, output_name + '_gene_list',
            ext='tsv', compression='gzip', index=False)

 ## Create Attribute List

In [ ]:
attribute_list = uf.createAttributeList(binary_matrix)
attribute_list.head()

In [ ]:
attribute_list.shape

In [ ]:
uf.saveData(attribute_list, path, output_name + '_attribute_list',
            ext='tsv', compression='gzip')

 ## Create Gene and Attribute Set Libraries

In [ ]:
uf.createUpGeneSetLib(binary_matrix, path, output_name + '_gene_up_set')

In [ ]:
uf.createUpAttributeSetLib(binary_matrix, path, 
                           output_name + '_attribute_up_set')

 ## Create Attribute Similarity Matrix

In [ ]:
attribute_similarity_matrix = uf.createSimilarityMatrix(binary_matrix.T, 'jaccard', np.bool)
attribute_similarity_matrix.head()

In [ ]:
uf.saveData(attribute_similarity_matrix, path,
            output_name + '_attribute_similarity_matrix', 
            compression='npz', symmetric=True, dtype=np.float32)

In [ ]:
# net.load_df(attribute_similarity_matrix.iloc[:,:].copy())
# net.filter_N_top('row', rank_type='sum', N_top=300)
# net.cluster()
# net.widget()

 ## Create Gene Similarity Matrix

In [ ]:
gene_similarity_matrix = uf.createSimilarityMatrix(binary_matrix, 'jaccard', np.bool)
gene_similarity_matrix.head()

In [ ]:
uf.saveData(gene_similarity_matrix, path, 
            output_name + '_gene_similarity_matrix',
            compression='npz', symmetric=True, dtype=np.float32)

 ## Create Gene-Attribute Edge List

In [ ]:
uf.createGeneAttributeEdgeList(binary_matrix, attribute_list, gene_list, 
                               path, output_name + '_gene_attribute_edge_list')

 # Create Downloadable Save File

In [ ]:
uf.createArchive(path)

 ### Link to download output files: [click here](./output_archive.zip)